In [10]:
from grid_trade import *
import pyupbit

In [4]:
# 지정 파라미터

MODE = "Geometric" # Arithmetic: 동일한 가격 차이 / Geometric: 동일한 가격 비율 차이
GRIDS = 5 # 그리드 수
COIN = "BTC" # 거래할 코인
BUDGET = 80000 # 사용할 예산
INTERVAL = 30 # 변동성 계산할 때 몇분 봉 사용
STD_NUM = 40 # 변동성 계산할 때 과거 몇개 사용
LOWER_STD = 3 # 하한선 변동성 배수

STOP_LOSS = 10 # 손절매 퍼센트
RESET_GRID = 5 # 계속 상승할 경우 그리드 리셋하는 퍼센트

In [6]:
key_path = "upbit_key.txt"

# trading bot 생성
bot = trading_bot(key_path)

연결성공
[{'currency': 'KRW', 'balance': '98408.49392041', 'locked': '0.0', 'avg_buy_price': '0', 'avg_buy_price_modified': True, 'unit_currency': 'KRW'}]


In [11]:
# 초기 환경 세팅
levels, diff, volume = bot.get_evn(COIN, MODE, GRIDS, BUDGET, INTERVAL, STD_NUM, LOWER_STD)

# 전체 주문 취소
bot.cancel_all_order(COIN)

# 초기 매수 주문
bot.levels_order(COIN, volume, levels)
print(bot.get_open_price(COIN))

while True:
    time.sleep(0.5) # 0.5초마다 업데이트
    
    current = pyupbit.get_current_price(f"KRW-{COIN}") # 현재 코인 가격
    avg_price = bot.get_avg(COIN) # 현재 평단가 가져오기
    
    
    # 수익률이 STOP_LOSS 보다 작을 경우 모두 시장가로 청산
    if avg_price != None:
        # 현재 가격과 비교해서 수익률 계산
        profit = (current-avg_price)/avg_price*100
        if profit < -STOP_LOSS:
            print("**STOP LOSS 실행**")
            print(f"{COIN} 현재가(KRW): {current}")
            print(f"{COIN} 평단가(KRW): {avg_price}")
            print(f"현재 손익: {profit:.2f}%")
            
            # 모든 주문 취소
            bot.cancel_all_order(COIN)
            # 모든 코인 시장가 매도
            bot.sell_all_market(COIN)
            
            print("종료")
            break
            
            
    # 가격이 일정 기준 이상 오르면 GRID 리셋
    if (current-levels[-1])/levels[-1]*100 > RESET_GRID:
        print("**GRID 리셋**")
        # 모든 주문 취소
        bot.cancel_all_order(COIN)

        # 현재 가격에 맞춰서 levels 다시 설정
        levels, diff, volume = bot.get_evn(COIN, MODE, GRIDS, BUDGET, INTERVAL, STD_NUM, LOWER_STD)

        # 초기 매수 주문 다시 넣기
        bot.levels_order(coin, volume, levels)
    
    
    # 미체결 주문수가 GRIDS와 다르면 추가 주문
    now_open = len(bot.upbit.get_order(f"KRW-{COIN}")) # 현재 미체결 주문수
    if GRIDS != now_open:
        balance_coin = bot.upbit.get_balance(COIN) # 코인 보유량으로 매수 매도 판단
        last_price = pyupbit.get_current_price(f"KRW-{COIN}") # 최근 체결 가격
        
        if balance_coin != 0: # 보유량이 0이 아니면 최근 체결은 매수
            side = "bid"
            if MODE == "Arithmetic": # MODE가 Arithmetic이면 목표가는 diff 만큼 더한 값
                price = last_price + diff 
            else: # MODE가 GEOMETRIC이면 매도 목표가는 diff 만큼 곱한 값
                price = last_price * diff 
            price = pyupbit.get_tick_size(price)
            ret = bot.upbit.sell_limit_order(f"KRW-{COIN}", price, volume)
            
        else: # 보유량이 0이면 최근 체결은 매도
            side = "ask"
            if MODE == "Arithmetic": # MODE가 Arithmetic이면 목표가는 diff 만큼 뺀 값
                price = last_price - diff 
            else: # MODE가 GEOMETRIC이면 매도 목표가는 diff 만큼 나눈 값
                price = last_price / diff 
            price = pyupbit.get_tick_size(price)
            ret = bot.upbit.buy_limit_order(f"KRW-{COIN}", price, volume)
        
        # 에러 발생시 message 출력
        if 'error' in ret.keys():
            print(ret['error']['message'])
            
        # 최근 체결 정보 표시
        else:
            print("")
            print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
            print(f"{side} -> {last_price}KRW  {volume}{COIN} 체결")
            print(bot.get_open_price(COIN))

2022-06-22 16:44:44

BTC 현재가(KRW): 26374000.0
호가단위(KRW): 1000
30분봉 40개 변동성: 468406.91
Lower Std: 3
Levels: [24968000, 25243000, 25521000, 25802000, 26086000]
Volume(BTC): 0.00060665
Difference(%): 1.10%
수익률: 1.10%
{'bid': ['24968000.0', '25243000.0', '25521000.0', '25802000.0', '26086000.0'], 'ask': []}


KeyboardInterrupt: 

In [13]:
# 전체 주문 취소
bot.cancel_all_order(COIN)